## Usando arquivo .env para controlar variaveis de ambiente
Para evitar exposição da chave `OPENAI_API_KEY` optei por utilizar arquivo `.env` com a informação da chave.

Para seguir o mesmo método basta criar um arquivo `.env` no mesmo diretório do arquivo `parent_rag.ipynb`.
A importação da chave será feita através da célula abaixo que faz a instalação de um biblioteca para carregar
os valores do arquivo `.env`.

In [12]:
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv

Note: you may need to restart the kernel to use updated packages.
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.storage import InMemoryByteStore
from langchain.retrievers import ParentDocumentRetriever
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_core.output_parsers import StrOutputParser


In [14]:
# LISTANDO AS PERGUNTAS A SEREM RESPONDIDAS
questions = [
    "Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?",
    "Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?",
    "Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?",
    "Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?",
    "Quais são os principais aspectos da crítica social e política presentes em \"Os Sertões\"? Como esses aspectos refletem a visão do autor sobre o Brasil da época?",
]

In [15]:
# Inicializando o modelo de embeddings
# Este modelo é usado para gerar representações vetoriais de textos
embeddings_model = OpenAIEmbeddings()

# Inicializando o modelo de linguagem (LLM)
# Aqui configuramos o nome do modelo e o número máximo de tokens na resposta
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",  # Nome do modelo de linguagem escolhido
    max_tokens=200,             # Limite de tokens para as respostas geradas
)

In [16]:
# Caminho do arquivo PDF
pdf_path = "os-sertoes.pdf"

# Inicializando o carregador de PDF
# Aqui estamos configurando para não extrair imagens do PDF
load_pdf = PyPDFLoader(pdf_path, extract_images=False)

# Carregando e dividindo o PDF em páginas
# Este método carrega o conteúdo do PDF e divide em uma lista de páginas
pages = load_pdf.load_and_split()

In [17]:

# Configuração do divisor para partes menores (filhos)
# Divide o texto em pedaços menores com tamanho máximo de 200 caracteres
child_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200  # Tamanho máximo de cada pedaço
)

# Configuração do divisor para partes maiores (pais)
# Divide o texto em pedaços maiores, com controle de sobreposição e outras configurações
parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,          # Tamanho máximo de cada pedaço
    chunk_overlap=200,        # Número de caracteres de sobreposição entre os pedaços
    length_function=len,      # Função para calcular o comprimento dos pedaços (aqui usamos `len`)
    add_start_index=True      # Adiciona o índice inicial de cada pedaço ao resultado
)


In [18]:
# Inicializando o armazenamento em memória
# Este armazenamento mantém os dados na memória durante a execução
store = InMemoryByteStore()

# Configurando o armazenamento vetorial
# Aqui, o armazenamento utiliza o modelo de embeddings fornecido
vectorstore = Chroma(embedding_function=embeddings_model)

In [19]:
# Configurando o recuperador de documentos principais (ParentDocumentRetriever)
# Este objeto será usado para gerenciar a recuperação de documentos com base em armazenamentos e divisores de texto

parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,       # Armazenamento vetorial para busca de documentos
    docstore=store,                # Armazenamento em memória para os documentos
    child_splitter=child_splitter, # Divisor para pedaços menores (filhos)
    parent_splitter=parent_splitter # Divisor para pedaços maiores (pais)
)

# Adicionando documentos ao recuperador
# O método abaixo insere as páginas carregadas no sistema, associando opcionalmente IDs a elas
parent_document_retriever.add_documents(pages, ids=None)

In [20]:
# Definindo o template de prompt
# Este template configura como o sistema interpretará o contexto e a pergunta
TEMPLATE = """
Você é um especialista em literatura brasileira. Responda a pergunta abaixo utilizando o contexto informado:

Contexto: {context}

Pergunta: {question}
"""

# Configurando o objeto de template para o prompt
# Ele utiliza as variáveis de entrada "context" e "question" para gerar o texto formatado
prompt = PromptTemplate(input_variables=["context", "question"], template=TEMPLATE)

# Configurando o analisador de saída (parser)
# O parser converte a saída do modelo em um formato estruturado
parser = StrOutputParser()

# Configurando a sequência executável
# A sequência define o fluxo de processamento:
# 1. O prompt é formatado
# 2. O modelo de linguagem (LLM) gera a resposta
# 3. O parser processa a saída final
sequence = RunnableSequence(prompt | llm | parser)


In [21]:
# Função para responder a uma pergunta com base no contexto fornecido
# A função utiliza o recuperador de documentos para obter o contexto e a sequência para gerar a resposta

def answer_question(question: str):
    # Recuperando o contexto relacionado à pergunta usando o recuperador de documentos
    context = parent_document_retriever.invoke(question)

    # Gerando a resposta utilizando o modelo de linguagem e o parser
    response = sequence.invoke({"context": context, "question": question})

    # Retornando a resposta gerada
    return response

In [22]:
# Iterando sobre as perguntas e obtendo as respostas
# 'enumerate' permite obter tanto o índice quanto a pergunta de cada item na lista
for index, question in enumerate(questions):
    # Obtendo a resposta para a pergunta atual utilizando a função 'answer_question'
    resposta = answer_question(question)
    # Exibindo o número da pergunta, a pergunta e a resposta gerada
    print({"numero": index, "pergunta": question, "resposta": resposta})

{'numero': 0, 'pergunta': 'Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?', 'resposta': 'Euclides da Cunha descreve o ambiente natural do sertão nordestino como uma região árida e hostil, marcada por extensos plainos ondulados e serranias. Ele destaca a dureza da natureza, com afloramentos gnáissicos e planaltos que se dobram, dificultando a vida dos habitantes locais. Essas características influenciam diretamente a vida dos habitantes, tornando a região desafiadora para a sobrevivência, com caudais de rios que revelam um pendor insensível para o norte e a presença de formações primitivas desaparecendo sob complexas séries de xistos metamórficos. Euclides da Cunha destaca a imponência dos quadros naturais do sertão nordestino, ressaltando como essas características influenc'}
{'numero': 1, 'pergunta': 'Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele r